University of Michigan

Master of Applied Data Science

SIADS699 - Capstone Project

Andre Onofre, Samantha Roska, Sawsan Allam

This Notebook: Machine Learning Model for Dermatoscopic Images

Augmentation used

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Import Libraries
import pandas as pd
import tensorflow as tf
import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import balanced_accuracy_score
import numpy as np
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_v2_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
from tensorflow.keras.applications.convnext import preprocess_input as convnext_preprocess

In [ ]:
# Directories
IMAGES_DIR = './Images/BCN20000-all-images-original/bcn_20k_train/'
IMAGES_ARRAY_DIR = './Images_Arrays/'
META_DATA_DIR = './Metadata/'
MODELS_DIR = './Models/'
TEST_IMAGES_DIR = './Images/Images_for_tests/'
TRAINING_RESULTS_DIR = './Training_Results/'

In [ ]:
# Load Pictures Array from file (should be (12412, 112, 112, 3) and (12412,8))

loaded_images_array = np.load(IMAGES_ARRAY_DIR + 'BCN20000_images_array (112_112).npy')
print(loaded_images_array.shape)

loaded_labels_array = np.load(IMAGES_ARRAY_DIR + 'BCN20000_labels_array (112_112).npy')
print(loaded_labels_array.shape)

In [ ]:
# Divide in Training and Test Set
X_train, X_test, y_train, y_test = train_test_split(loaded_images_array, loaded_labels_array, test_size=0.2, random_state=42,stratify=loaded_labels_array)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Calculate Class Weights based on training set
y_integers = np.argmax(y_train, axis=1)
class_weights = compute_class_weight('balanced', classes=np.unique(y_integers), y=y_integers)
d_class_weights = dict(enumerate(class_weights))
for class_idx, weight in d_class_weights.items():
    print(f'Class {class_idx}: {weight:.2f}')

# Propagating to all samples
final_weights_train = np.array([d_class_weights[label] for label in y_integers])


In [ ]:
# Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1)
  ])

def augment_data(image, label, weight=None):
    image = data_augmentation(image, training=True)
    if weight is not None:
      return image, label, weight
    else:
      return image, label

In [ ]:
def train_run_model(model_type,
                    X_train,
                    X_test,
                    y_train,
                    y_test,
                    Use_Class_Weights,
                    Class_Weights,
                    Use_Augmentation):

  print('------------------------------------------------------------------')
  print('\nTraining Model ', model_type)

  # Load base model
  print('\nLoading Base Model...')
  model_string = "tf.keras.applications." + model_type + "(input_shape=(112,112,3),include_top=False,weights='imagenet')"
  base_model = eval(model_string)
  print('Base Model Loaded!')
  base_model.trainable = False

  # Define the number of layers to train
  total_layers = len(base_model.layers)
  number_of_layers_not_to_train = int(0.2 * total_layers)
  base_model.trainable = True
  for layer in base_model.layers[0:number_of_layers_not_to_train]:
    layer.trainable = False

  trainable_layer_count = 0
  for layer in base_model.layers:
    if layer.trainable:
        trainable_layer_count += 1

  print('\nTotal Layers: ', total_layers)
  print('Layers to Train: ', total_layers - number_of_layers_not_to_train)
  print('Layers marked as Trainable: ', trainable_layer_count)

  # Include Top Layers (flatten, dense, softmax)
  print('\nAdding Top Layers...')
  last_desired_layer = base_model.get_layer(base_model.layers[-1].name)
  last_output = last_desired_layer.output
  new_output = layers.Flatten()(last_output)
  new_output = layers.Dense(512,activation='relu')(new_output)
  new_output = layers.Dropout(0.2)(new_output)
  new_output = layers.Dense(8, activation = 'softmax')(new_output)
  final_model = Model(inputs=base_model.input, outputs=new_output)
  print('Model Architecture Built!')

  # Trainable parameters
  trainable_params = sum([tf.size(w).numpy() for w in final_model.trainable_weights])
  print(f"\nTrainable Parameters: {trainable_params:,}")

  # Non-trainable parameters
  non_trainable_params = sum([tf.size(w).numpy() for w in final_model.non_trainable_weights])
  print(f"Non-Trainable Parameters: {non_trainable_params:,}")

  # Total number of parameters (Trainable + Non-trainable)
  total_params = final_model.count_params()
  print(f"Total Parameters: {total_params:,}")

  # Normalization
  if model_type == 'EfficientNetB2':
    X_train = efficientnet_preprocess(X_train)
    X_test = efficientnet_preprocess(X_test)
    print('\nNormalization done for EfficientNetB2')
  elif model_type == 'EfficientNetB0':
    X_train = efficientnet_preprocess(X_train)
    X_test = efficientnet_preprocess(X_test)
    print('\nNormalization done for EfficientNetB0')
  elif model_type == 'DenseNet121':
    X_train = densenet_preprocess(X_train)
    X_test = densenet_preprocess(X_test)
    print('\nNormalization done for DenseNet121')
  elif model_type == 'MobileNetV2':
    X_train = mobilenet_v2_preprocess(X_train)
    X_test = mobilenet_v2_preprocess(X_test)
    print('\nNormalization done for MobileNetV2')
  elif model_type == 'ConvNeXtTiny':
    X_train = convnext_preprocess(X_train)
    X_test = convnext_preprocess(X_test)
    print('\nNormalization done for ConvNeXtTiny')
  else:
    print('\nNormalization not done')

  # Prepare final dataset
  BUFFER_SIZE = len(X_train)
  BATCH_SIZE = 128

  # Use Class Weights or not
  if Use_Class_Weights:
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train, Class_Weights))
    print('\nClass Weights Enabled')
  else:
    print('\nClass Weights Disabled')
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

  # Augmentation
  if Use_Augmentation:
    train_dataset = train_dataset.map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)
    print('\nAugmentation Enabled')
  else:
    print('\nAugmentation Disabled')

  # Shuffle samples before each epoch
  train_dataset = train_dataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

  # Compile Model and Train
  print('\nCompiling Model...')
  Adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
  final_model.compile(optimizer = Adam_optimizer,loss = 'categorical_crossentropy', metrics = ['accuracy',tf.keras.metrics.F1Score(average='weighted', name='f1_score')])
  lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_f1_score',factor=0.1,patience=10, mode='max',verbose=1)
  callback = [keras.callbacks.EarlyStopping(monitor='val_f1_score',patience=30, mode='max',restore_best_weights=True), lr_schedule]
  print('Model Compiled')

  print('\nTraining Model...')
  history = final_model.fit(train_dataset,epochs = 200,verbose = 1, validation_data=(X_test, y_test), callbacks=[callback])

  # Evaluate Model
  print('-------------------------------------------')
  print('Results for ' + model_type)
  print('-------------------------------------------')
  print('\nEvaluation')
  test_loss, test_accuracy, f1_score = final_model.evaluate(X_test, y_test, verbose=0)
  test_accuracy = round(test_accuracy,2)
  print('Test Accuracy: ', test_accuracy)
  f1_score = round(f1_score,2)
  print('F1 Score: ', f1_score)

  #  Balanced Score
  y_pred = final_model.predict(X_test)
  y_pred_classes = np.argmax(y_pred, axis=1)
  y_test_classes = np.argmax(y_test, axis=1)
  balanced_accuracy = round(balanced_accuracy_score(y_test_classes, y_pred_classes),2)
  print('Balanced accuracy: ', balanced_accuracy)

  # Accuracy per Class
  print('\nAccuracy Per Class')
  cm = confusion_matrix(y_test_classes, y_pred_classes)
  true_positives = np.diag(cm)
  actual_instances_per_class = np.sum(cm, axis=1)
  accuracy_per_class = np.divide(true_positives, actual_instances_per_class)
  for i, acc in enumerate(accuracy_per_class):
    print(f"Class {i}: {acc*100:.1f}% (Samples: {actual_instances_per_class[i]})")
  accuracy_per_class_rounded = [round(x, 2) for x in accuracy_per_class]

  # Plot Confusion Matrix
  print('\nConfusion Matrix')
  cm_display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1, 2, 3, 4, 5, 6, 7])
  cm_display.plot()
  plt.show()

  # Save Model
  mmodel_name = MODELS_DIR + 'BCN20000-V103-' + model_type + '.keras'
  print('\nSaving Model...')
  final_model.save(mmodel_name)
  print('Model Saved!')

  return test_accuracy, balanced_accuracy, f1_score, *accuracy_per_class_rounded

In [ ]:
# Dataset to store results
data_schema = {
    'Model': pd.Series(dtype='str'),
    'Accuracy': pd.Series(dtype='float'),
    'Balanced_Accuracy': pd.Series(dtype='float'),
    'F1': pd.Series(dtype='float'),
    'Acc_0': pd.Series(dtype='float'),
    'Acc_1': pd.Series(dtype='float'),
    'Acc_2': pd.Series(dtype='float'),
    'Acc_3': pd.Series(dtype='float'),
    'Acc_4': pd.Series(dtype='float'),
    'Acc_5': pd.Series(dtype='float'),
    'Acc_6': pd.Series(dtype='float'),
    'Acc_7': pd.Series(dtype='float')
}

df_results = pd.DataFrame(data_schema)

In [ ]:
# Train models
models_to_train = ['EfficientNetB2', 'EfficientNetB0','DenseNet121', 'MobileNetV2', 'ConvNeXtTiny']

for model_to_train in models_to_train:
  results_list = []
  test_accuracy, balanced_accuracy, f1_score, acc_0, acc_1, acc_2, acc_3, acc_4, acc_5, acc_6, acc_7  = train_run_model(model_to_train,
                                                                                                        X_train.copy(),
                                                                                                        X_test.copy(),
                                                                                                        y_train,
                                                                                                        y_test,
                                                                                                        Use_Class_Weights=False,
                                                                                                        Class_Weights=final_weights_train,
                                                                                                        Use_Augmentation=True)

  # Record Results on dataset
  new_record_data = {
    'Model': model_to_train,
    'Accuracy': test_accuracy,
    'Balanced_Accuracy': balanced_accuracy,
    'F1': f1_score,
    'Acc_0': acc_0,'Acc_1': acc_1,'Acc_2': acc_2, 'Acc_3': acc_3, 'Acc_4': acc_4, 'Acc_5': acc_5, 'Acc_6': acc_6, 'Acc_7': acc_7}
  results_list.append(new_record_data)
  new_record_df = pd.DataFrame(results_list)
  df_results = pd.concat([df_results, new_record_df], ignore_index=True)

# Save Results
df_results.to_csv(TRAINING_RESULTS_DIR + 'BCN20000-V103-Training_Results.csv', index=False)
print('\nResults Saved')

In [ ]:
df_results